In [ ]:
import numpy as np
import pandas as pd
import regex as re
from bs4 import BeautifulSoup
import contractions
import unidecode
import re
import pickle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
import string
from sklearn.utils import shuffle
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification,RobertaForSequenceClassification,RobertaTokenizer
from transformers import EarlyStoppingCallback
# from gensim.parsing.preprocessing import remove_stopwords
from transformers import XLMRobertaModel, XLMRobertaTokenizer
import os


In [ ]:

def removeHtml(sentence):
    soup = BeautifulSoup(sentence, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

def removePuncu(sentence):
    res = re.sub(r'[^\w\s]', '', sentence)
    return res

def removeLink(sentence):
    result = re.sub(r'http\S+', '', sentence)
    return result

def removeWhiteSpaces(sentence):
    sentence = sentence.strip()
    return " ".join(sentence.split())

def expand_contractions(sentence):
    sentence = contractions.fix(sentence)
    return sentence

def removeAccented(sentence):
    sentence = unidecode.unidecode(sentence)
    return sentence

def removeSpec(sentence):
    sentence = re.sub("[^A-Z]", " ", sentence, 0, re.IGNORECASE)
    return sentence

def removeNum(sentence):
    pattern = r'[0-9]'
    sentence = re.sub(pattern, '', sentence)
    return sentence

def lowerCase(sentence):
    sentence = sentence.lower()
    return sentence


# with open(r'C:\Users\LEGION\Documents\Untitled Folder 2\Emoji_Dict.p', 'rb') as fp:
#     Emoji_Dict = pickle.load(fp)
# Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

# def convertEmoji(sentence):
#     for emot in Emoji_Dict:
#         sentence = re.sub(r'('+emot+')', "".join(Emoji_Dict[emot].replace(",","").replace(":","").split()),sentence)
#     return sentence

def add_labels(value):
    if value == 'positive':
        return 2
    elif value == 'negative':
        return 0
    elif value == 'neutral':
        return 1
    

In [ ]:
def preprocessing(data):
    sent_or_word = 'translation_sentence_level'
    data[sent_or_word] = data[sent_or_word].apply(lambda x: lowerCase(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removePuncu(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeSpec(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeHtml(x))

    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeAccented(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeLink(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeNum(x))

    data[sent_or_word] = data[sent_or_word].apply(lambda x: expand_contractions(x))
#     da[sent_or_word] = da[sent_or_word].apply(lambda x: convertEmoji(x))
    data[sent_or_word] = data[sent_or_word].apply(lambda x: removeWhiteSpaces(x))
    # data[sent_or_word] = data[sent_or_word].apply(lambda x: removeUser(x))
    x = data[sent_or_word].to_list()
    y = list(data['categorical_labels']) 
    
    return x,y


In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
model_results_dict = {"Model_Name":[], "Accuracy": [], 'Recall': [], 'Precision': [], "f1":[]}

In [ ]:
path = "modelMergedRemoveEmojiTwitterRoberta/twitter-roberta-base-sentiment_Train_merged_Twitter_Roberta_Removed_Emoji"
# modelMergedRemoveUserTwitterRoberta/twitter-roberta-base-sentiment_Train_merged_Twitter_Roberta_Removed_User
# modelMergedRemovedUserRoberta/roberta-base_Train_merged_Roberta_Removed_User
max_length = 128
num_labels = 3
task = 'sentiment'
# modelName = "roberta-base"
modelName = f"cardiffnlp/twitter-roberta-base-{task}"

# safe_model_name = sanitize(model_name)

tokenizer = AutoTokenizer.from_pretrained(modelName)

model = AutoModelForSequenceClassification.from_pretrained(path)


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment/resolve/main/config.json from cache at /home/ablstation2/.cache/huggingface/transformers/7dd97280b5338fb674b5372829a05a1aaaa76f9f2fa71c36199f2ce1ee1104a0.4c7ca95b4fd82b8bbe94fde253f5f82e5a4eedefe6f86f6fa79efc903d6cfe60
Model config RobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-roberta-base-sentiment",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "l

In [ ]:

test_data = pd.read_csv("emojiRemovedFiles/MergedEmojiRemoved/testDataMerged.tsv", sep = '\t')
test_data = test_data[test_data['translation_sentence_level'].notna()]
test_data['categorical_labels'] = test_data['label'].map(add_labels)
X_test, y_test = preprocessing(test_data)

X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=max_length,add_special_tokens=True)
test_dataset = Dataset(X_test_tokenized)

In [ ]:
def compute_metrics(p):
        pred, labels = p
        pred = np.argmax(pred, axis=1)
        accuracy = accuracy_score(y_true=labels, y_pred=pred)
        recall = recall_score(y_true=labels, y_pred=pred,average='weighted')
        precision = precision_score(y_true=labels, y_pred=pred,average='weighted')
        f1 = f1_score(y_true=labels, y_pred=pred,average='weighted')

        return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
newModelNameForSaving = modelName.split("/")[-1]
x = "Test_mergedTwitterRobertaRemovedEmoji.tsv"

In [ ]:
args = TrainingArguments(
    output_dir="checkpointsTestMerRemUser/{}_{}".format(newModelNameForSaving,x.split(".")[0]),
    evaluation_strategy="epoch",
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    num_train_epochs=5,
    seed=7,
    load_best_model_at_end=True,
    save_strategy='epoch',
    overwrite_output_dir=True
      #save_strategy = "no"
  )

trainer = Trainer(
      model=model,
      args=args,
      compute_metrics=compute_metrics,
      callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
  )


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
100%|██████████| 23/23 [01:53<00:00,  4.95s/it]


In [ ]:
predictions = trainer.predict(test_dataset)

pred = np.argmax(predictions.predictions, axis=-1)

  
accuracy = accuracy_score(y_true=y_test, y_pred=pred)
recall = recall_score(y_true=y_test, y_pred=pred,average='weighted')
precision = precision_score(y_true=y_test, y_pred=pred,average='weighted')
f1 = f1_score(y_true=y_test, y_pred=pred,average='weighted')

print(modelName)
print("accuracy: {}".format(accuracy))
print("recall: {}".format(recall))
print("precision: {}".format(precision))
print("f1: {}".format(f1))


model_results_dict['Model_Name'].append(x)
model_results_dict['Accuracy'].append(accuracy)
model_results_dict['Recall'].append(recall)
model_results_dict['Precision'].append(precision)
model_results_dict['f1'].append(f1)

df = pd.DataFrame(data = model_results_dict)
df.to_csv("{}_MergedDataTwitterRobertaRemovedEmojiTestResults.csv".format(newModelNameForSaving))

***** Running Prediction *****
  Num examples = 11647
  Batch size = 256
/home/ablstation2/.pyenv/versions/3.6.5/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
100%|██████████| 23/23 [00:05<00:00,  4.16it/s]

cardiffnlp/twitter-roberta-base-sentiment
accuracy: 0.66429123379411
recall: 0.66429123379411
precision: 0.6731019432438173
f1: 0.661372015086978
